In [6]:
dict(
    author="Rob Stenson",
    title="Blender",
    date="1/9/2023",
)

{'author': 'Rob Stenson', 'title': 'Blender', 'date': '1/9/2023'}

In [5]:
#hide-publish
from coldtype.notebook import *

Behind the scenes, Coldtype uses the Skia library to rasterize two-dimensional vectors. But what if we want to rasterize *three-dimensional* graphics? One option is to use Blender.


Because Blender has an incredible Python API, it's not too difficult to use it programmatically — i.e. to write a normal Coldtype script, mark a few things (with metadata specific to Blender), and then let Blender & Coldtype take care of the translation to three dimensions. Here's an example:

In [4]:
from coldtype.blender import *

fnt = Font.Find("SwearCilatiVariable")

@b3d_animation(timeline=60)
def varfont(f):
    return (Glyphwise("Vari", lambda g:
        Style(fnt, 325,
            opsz=f.adj(-g.i*5).e("seio", 1, rng=(0.98, 0)),
            wght=f.adj(-g.i*15).e("seio", 1, rng=(0.98, 0))
            ))
        .align(f.a.r)
        .mapv(lambda i, p: p
            .ch(b3d(lambda bp: bp
                .extrude(f.adj(-i*5)
                    .e("ceio", 1, rng=(0.015, 3)))))))

## Running Code in Blender

First off, your virtual environment needs to be built with the same version of Python as the Blender version you’re targeting.

Behind the scenes, Coldtype uses a tool called `b3denv` to communicate with Blender, and `b3denv` — installed as a dependency of Coldtype — makes it easy to create a virtual environment using the Python bundled with Blender.

```bash
b3denv python -m venv benv
```

Now you’ll have a blender-specific virtual environment named `benv`, which you can activate the usual way, `source benv/scripts/activate` on Mac.

To get a Blender window to show up, all you need to do is use the ``@b3d_animation`` decorator in place of the standard @animation decorator, and add ``-bw 1`` to the command-line invocation. Or, if you want a set of sensible CLI defaults, try ``-p b3d`` instead, which stands for ``--profile=b3d`` and sets ``-bw 1`` as part of some other settings in the b3d profile.

So, to use an example from the Coldtype repo, you could save the code from above and run:

```bash
coldtype examples/blender/varfont.py -p b3d
```

This should launch both a standard Coldtype window (with a 2D Skia renderer) and a Blender GUI window, which should automatically render the same thing as the 2D window, except in 3D. Put another way: you do not need to open Blender yourself, since Coldtype launches it as a background process (necessary to connect the live-code-reloading part of Coldtype to Blender). To quit both Coldtype and Blender, just hit ctrl-c in the terminal.

What's different in Blender is that the contents of the scene aren’t re-created from scratch every time you render; instead, you annotate specific elements in your returned result, then those annotated results are displayed in Blender, as persistent objects. This means you can use Blender in a hybrid fashion, creating objects using the GUI, saving the file, and then re-saving your Coldtype source file for automatic updates in Blender itself.

If you’d like to skip seeing the 2D Coldtype window, you can use `--b3dlo` instead. 